# Unlocking AI Agents for Network Engineering

Structure:

* Libraries + Setup
* First Agent
* First custom Tool, Function/Tool Calling
* Structured Output with Ollama
* 

https://gorilla.cs.berkeley.edu/leaderboard.html